In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as  pd
import sklearn
import os
import time
from tensorflow import keras
#答应版本
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
#导入数据
(x_train_all,y_train_all),(x_test,y_test) = keras.datasets.fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]
print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
#图片归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaler = scaler.fit_transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaler = scaler.fit_transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [4]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [5]:
#tf.keras.Sequential()
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

#relu : y  = max(0,x)
#softmax: 将向量变成概率分布 x = [x1,x2,x3] y = [e^x1/sum,e^x2/sum,e^x3/sum] sum = e^x1+e^x2+e^x

#loss 如果lable 是一个one-hot 向量 ：categorical_croossentropy，不是向量：sparse_categorical_crossentropy
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = 'sgd',
             metrics = ["accuracy"])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [25]:
logdir = r'C:\Users\Xiaoi\Desktop\tensorflow2.0\callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")

_callbacks = [
    keras.callbacks.EarlyStopping(patience=10) # 5轮之内，最低为1e-3 就停止
]
model.fit(x_train_scaled,
                    y_train,
                   epochs=10,
                   validation_data=(x_valid_scaler,y_valid),
                   callbacks=_callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 3s 61us/sample - loss: 0.3862 - accuracy: 0.8633 - val_loss: 0.3890 - val_accuracy: 0.8646
Epoch 2/10
55000/55000 [==============================] - 3s 60us/sample - loss: 0.3796 - accuracy: 0.8660 - val_loss: 0.3853 - val_accuracy: 0.8654
Epoch 3/10
55000/55000 [==============================] - 3s 61us/sample - loss: 0.3736 - accuracy: 0.8683 - val_loss: 0.3802 - val_accuracy: 0.8662
Epoch 4/10
55000/55000 [==============================] - 3s 60us/sample - loss: 0.3680 - accuracy: 0.8699 - val_loss: 0.3747 - val_accuracy: 0.8664
Epoch 5/10
55000/55000 [==============================] - 3s 60us/sample - loss: 0.3625 - accuracy: 0.8720 - val_loss: 0.3744 - val_accuracy: 0.8704
Epoch 6/10
55000/55000 [==============================] - 3s 59us/sample - loss: 0.3579 - accuracy: 0.8726 - val_loss: 0.3731 - val_accuracy: 0.8702
Epoch 7/10
55000/55000 [==============================] -

In [29]:
MODEL_NAME = "./my_fashion_mnist"
import time

model_version = int(time.time())
model_path = os.path.join(MODEL_NAME, str(model_version))
os.makedirs(model_path)

In [30]:
for root, dirs, files in os.walk(MODEL_NAME):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_fashion_mnist/
    1563155107/


In [31]:
keras.experimental.export_saved_model(model,model_path)

W0715 09:45:08.684412 16604 tf_logging.py:161] Export includes no default signature!
W0715 09:45:09.082137 16604 tf_logging.py:161] Export includes no default signature!


In [32]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'eval' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: init_1
  Method name is: 

signature_def['eval']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_2_target'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, -1)
        name: dense_2_target:0
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['loss'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: loss/mul:0
    outputs['metrics/accuracy/update_op'] tensor_info:
        dtype: DT_FLOAT
        shape: (0

In [33]:
# Load the saved keras model back.
new_model = tf.keras.experimental.load_from_saved_model(model_path)
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [35]:
X_new = x_test_scaler[:3]
np.save("my_fashion_mnist_tests.npy", X_new, allow_pickle=False)

In [36]:
input_name = model.input_names[0]
input_name

'flatten_input'

In [37]:
!saved_model_cli run --dir {model_path} 
                     --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_fashion_mnist_tests.npy

Result for output key dense_2:
[[1.28984129e-05 2.18171340e-06 2.95067748e-05 8.24837389e-06
  2.09156351e-05 8.10407102e-02 9.42812585e-06 1.70225546e-01
  1.51904393e-03 7.47131526e-01]
 [1.78754824e-04 1.26196655e-05 9.82687175e-01 2.08884430e-05
  5.05574839e-03 9.16085909e-08 1.20141432e-02 5.03308906e-09
  3.03181641e-05 2.03872304e-07]
 [4.83477379e-06 9.99965906e-01 8.04422325e-06 1.52600005e-05
  5.38615222e-06 9.29826527e-08 5.03960962e-08 1.49875959e-07
  3.34067039e-07 2.07831863e-09]]


2019-07-15 09:53:45.987219: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
W0715 09:53:45.989095 12576 deprecation.py:323] From c:\users\xiaoi\desktop\tensorflow2.0\venv\lib\site-packages\tensorflow\python\tools\saved_model_cli.py:339: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0715 09:53:46.016094 12576 deprecation.py:323] From c:\users\xiaoi\desktop\tensorflow2.0\venv\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:


# TensorFlow Serving

Install [Docker](https://docs.docker.com/install/) if you don't have it already. Then run:

```bash
docker pull tensorflow/serving

docker run -it --rm -p 8501:8501 \
   -v "`pwd`/my_fashion_mnist:/models/my_fashion_mnist" \
   -e MODEL_NAME=my_fashion_mnist \
   tensorflow/serving
```

Once you are finished using it, press Ctrl-C to shut down the server.

In [40]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})
print(input_data_json[:200] + "..." + input_data_json[-200:])

{"signature_name": "serving_default", "instances": [[[-0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831,...731323, -0.16853250563144684, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831, -0.8138857483863831]]]}


In [42]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_fashion_mnist:predict'
            
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/my_fashion_mnist:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E5951A5A58>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))